In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.utils import class_weight as cw, resample
import random

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import xlrd
all_files_loc  = r'/content/gdrive/My Drive/DATASET'
all_files = os.listdir(all_files_loc)

In [0]:
Features ="Min, Max, STD, Mean, Median, Activity, Mobility, Complexity, Kurtosis, 2nd Difference Mean, 2nd Difference Max, 1st Difference Mean, 1st Difference Max, Coeffiecient of Variation, Skewness, Wavelet Approximate Mean, Wavelet Approximate Std Deviation, Wavelet Detailed Mean, Wavelet Detailed Std Deviation, Wavelet Approximate Energy, Wavelet Detailed Energy, Wavelet Approximate Entropy, Wavelet Detailed Entropy, Mean of Vertex to Vertex Slope, Var of Vertex to Vertex Slope, FFT Delta Max Power, FFT Theta Max Power, FFT Alpha Max Power, FFT Beta Max Power, Delta/Alpha, Delta/Theta"
features=Features.split(", ")

In [0]:
chs = ['FP1','FPZ','FP2','AF3','AF4','F7','F5','F3','F1','FZ','F2','F4','F6','F8','FT7','FC5','FC3','FC1','FCZ','FC2','FC4','FC6','FT8','T7','C5','C3','C1','CZ','C2','C4','C6','T8','M1','TP7','CP5','CP3','CP1','CPZ','CP2','CP4','CP6','TP8','M2','P7','P5','P3','P1','PZ','P2','P4','P6','P8','PO7','PO5','PO3','POZ','PO4','PO6','PO8','O1','OZ','O2']

In [0]:
import pandas as pd
import numpy as np
wb = xlrd.open_workbook(all_files_loc+'/Data_4_Import_REST.xlsx')
sheet = wb.sheet_by_index(0)
allSeshRows=list()
for i in range(1,sheet.nrows):
    row = sheet.row_values(i)
    allSeshRows.append(row[6])

In [0]:
fullDataset=pd.DataFrame()

In [0]:
unwanted_numbers=[522,581,539,571,572,613]
subject_IDs=np.arange(507, 628, 1).tolist()
subject_IDs = [num for num in subject_IDs if num not in unwanted_numbers] 

In [0]:
labels=pd.DataFrame(allSeshRows,index=subject_IDs)

In [0]:
for filename in all_files :
  if filename.endswith('.npy'):
      data1 = np.load(all_files_loc+'/'+filename, allow_pickle=True)
      df0 = pd.DataFrame(data1[0][0], columns=features, index=chs)
      df = df0.reset_index().copy()
      df.rename(columns={'index': 'channel'}, inplace=True)
      test = df.loc[df['channel'] == 'T8']
      test = test.drop(['channel'], axis=1)
      fullDataset = fullDataset.append(test.iloc[:][:],ignore_index=True)

In [0]:
fullDataset=fullDataset[['1st Difference Max','Wavelet Detailed Energy', 'Var of Vertex to Vertex Slope', 'Wavelet Detailed Std Deviation','Delta/Alpha', 'Delta/Theta','Kurtosis']]

In [0]:
scaler = MinMaxScaler()
fullDataset.index=subject_IDs
fullDataset=fullDataset.reindex(subject_IDs)
fullDataset[:]=scaler.fit_transform(fullDataset[:])

In [0]:
fullDataset=fullDataset[['1st Difference Max','Wavelet Detailed Energy', 'Var of Vertex to Vertex Slope', 'Wavelet Detailed Std Deviation','Delta/Alpha', 'Delta/Theta','Kurtosis']]
fullDataset= fullDataset.assign(BDI=labels[0])

In [0]:
fullDataset['BDI']=pd.cut(fullDataset['BDI'],bins=[0,13.0,19.0,28.0,63.0],labels=[0,1,2,3])
fullDataset['BDI'] = fullDataset['BDI'].fillna(0)

In [0]:
def upsample(classNum,sampleSize):
 return resample(classNum,
  replace=True,    
  n_samples=sampleSize,     
  random_state=123) 


In [0]:
class0= fullDataset[fullDataset.BDI==0]
class1= fullDataset[fullDataset.BDI==1]
class2= fullDataset[fullDataset.BDI==2]
class3= fullDataset[fullDataset.BDI==3]
majority_count=len(class0)

In [0]:
upsampled_class1=upsample(class1,majority_count)
upsampled_class2=upsample(class2,majority_count)
upsampled_class3=upsample(class3,majority_count)

In [18]:
print(len(class0))
print(len(upsampled_class1))
print(len(upsampled_class2))
print(len(upsampled_class3))

73
73
73
73


In [0]:
upsampledData=pd.concat([class0,upsampled_class1,upsampled_class2,upsampled_class3])

In [0]:
X=upsampledData[['1st Difference Max','Wavelet Detailed Energy', 'Var of Vertex to Vertex Slope', 'Wavelet Detailed Std Deviation','Delta/Alpha', 'Delta/Theta','Kurtosis']]
y=upsampledData[['BDI']]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True)

In [0]:
params_grid = [{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}]

In [0]:
svm=GridSearchCV(estimator=SVC(kernel='rbf', probability=True), param_grid=params_grid, cv=5,return_train_score=True)
fitted=svm.fit(X_train, y_train.values.ravel())
scores=fitted.cv_results_

In [0]:
classifier = OneVsRestClassifier(svm.best_estimator_)
fitted=classifier.fit(X_train, y_train)

In [0]:
predicted = classifier.predict(X_test)
predicted_probability = classifier.predict_proba(X_test)

In [26]:
from sklearn.metrics import classification_report
target_names=['minimal range','mild','moderate','severe']
print(classification_report(y_test,predicted,target_names=target_names))

               precision    recall  f1-score   support

minimal range       0.34      0.41      0.37        27
         mild       0.00      0.00      0.00        26
     moderate       0.00      0.00      0.00        18
       severe       0.29      0.73      0.42        26

     accuracy                           0.31        97
    macro avg       0.16      0.28      0.20        97
 weighted avg       0.17      0.31      0.22        97



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.metrics import confusion_matrix

In [28]:
confMat=confusion_matrix(y_true=y_test,y_pred=predicted,labels=[0,1,2,3])

print(confMat)

[[11  0  0 16]
 [ 6  0  0 20]
 [ 8  0  0 10]
 [ 7  0  0 19]]


In [29]:
total=sum(sum(confMat))
TP=confMat[0,0]
FP=confMat[1,0]
FN=confMat[0,1]
TN=confMat[1,1]
sensitivity = TP/(TP+FP)
print('Sensitivity : ', sensitivity )
specificity = TN/(FP+TN)
print('specificity : ', specificity )

Sensitivity :  0.6470588235294118
specificity :  0.0


In [0]:
predicted_probability=pd.DataFrame(predicted_probability)
predicted_probability.columns=['probability of class1','probability of class2','probability of class3','probability of class4']
